In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Fri Jun 10 19:46:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 32%   41C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 35%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de
    
    db_rm_us_2013 = db.rm_patent_us_2013
    db_rm_cn_2013 = db.rm_patent_cn_2013
    db_rm_de_2013 = db.rm_patent_de_2013

except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_rm_us_2013

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2013')

# DE-CN-US

## 2013

### DE

In [6]:
data_assignee = db_rm_de_2013.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [7]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_2013.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Robert Bosch GmbH,H01L2224,3
1,Robert Bosch GmbH,H05B3,2
2,Robert Bosch GmbH,G01N27,2
3,Robert Bosch GmbH,H01L35,2
4,Robert Bosch GmbH,30,2


In [8]:
df_cpc.shape

(70, 3)

In [9]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [10]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [11]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,B05,D06,G01,G06,H01,H02,H04,H05,Y02
0,Armstrong DLW GmbH,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Electronics and Telecommunications Research In...,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0
2,IOT - Innovative Oberflachentechnologien GmbH,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IOT INNOVATIVE OBERFLACHENTECHNOLOGIEN GmbH,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Intel IP Corp,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [12]:
final_table_de.shape

(7, 10)

In [13]:
final_table_de.to_csv('rm_company_cpc_de_2013.csv',index=0)

把final_table資料存進MongoDB

In [14]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_de_2013

In [15]:
# title
conn_db_collection_rm.insert_one({"company":final_table_de.columns.tolist()[0],"cpc":final_table_de.columns.tolist()[1:]})

# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### CN

In [17]:
data_assignee = db_rm_cn_2013.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [18]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn_2013.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,9
1,Jiangsu IoT Research and Development Center,66325,6
2,Jiangsu IoT Research and Development Center,07,3
3,Jiangsu IoT Research and Development Center,H01L24,3
4,Jiangsu IoT Research and Development Center,66333,3


In [19]:
df_cpc.shape

(1893, 3)

In [20]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [21]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [22]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A23,A43,A47,A61,B01,B09,B21,B22,B23,...,G10,G11,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,A Mo Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ABRAM SCIENT Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ANHUI LONGCOM INTERNET OF THINGS Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
final_table_cn.shape

(187, 58)

In [24]:
final_table_cn.to_csv('rm_company_cpc_cn_2013.csv',index=0)

把final_table資料存進MongoDB

In [25]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_cn_2013

In [26]:
# title
conn_db_collection_rm.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### US

In [27]:
data_assignee = db_rm_us_2013.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [28]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_us_2013.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Cisco Technology Inc,H04L45,40
1,Cisco Technology Inc,H04L41,31
2,Cisco Technology Inc,H04L43,22
3,Cisco Technology Inc,H04W40,16
4,Cisco Technology Inc,16,9


In [29]:
df_cpc.shape

(1364, 3)

In [30]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [31]:
# 轉成company-cpc 的總數表格
table_us = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [32]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,B01,B29,B32,B65,B81,C10,G01,G02,G03,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,Aeris Communications Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
1,Alcatel Lucent SAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,21.0,0.0,1.0,0.0,0.0
2,Amazon Technologies Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,Arizona Board of Regents of ASU,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0


In [33]:
final_table_us.shape

(68, 25)

In [34]:
final_table_us.to_csv('rm_company_cpc_us_2013.csv',index=0)

把final_table資料存進MongoDB

In [35]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_2013

In [36]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### 合併

In [37]:
# concat de cn us
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2.head()

cpc_n,company,A23,A43,A47,A61,B01,B05,B09,B21,B22,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,Armstrong DLW GmbH,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Electronics and Telecommunications Research In...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,IOT - Innovative Oberflachentechnologien GmbH,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IOT INNOVATIVE OBERFLACHENTECHNOLOGIEN GmbH,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Intel IP Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0


In [38]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2.head()

cpc_n,company,A23,A43,A47,A61,B01,B05,B09,B21,B22,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,A Mo Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ABRAM SCIENT Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ANHUI LONGCOM INTERNET OF THINGS Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
df_c2.to_csv('rm_company_cpc_all_2013.csv',index=0)

In [40]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_all_2013

In [41]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})